In [5]:
import mujoco
#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
import mediapy as media
import matplotlib.pyplot as plt

import mujoco.viewer
from pathlib import Path


# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

model_dir = Path("/home/anton/master/mujoco_menagerie/universal_robots_ur5e")
model_xml = model_dir / "scene.xml"


xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_path(str(model_xml))
data = mujoco.MjData(model)

m = model
d = data

path = np.arange(-1, -2, -0.001)
i = 0
end = len(path)


with mujoco.viewer.launch_passive(m, d) as viewer:
  # Close the viewer automatically after 30 wall-seconds.
  start = time.time()
  while viewer.is_running(): #and time.time() - start < 30:
    step_start = time.time()

    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_forward(m, d)

    # Example modification of a viewer option: toggle contact points every two seconds.
    with viewer.lock():
      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(d.time % 2)
    
    if i < end:
      pos = path[i]
      print(path[i]) 
      d.qpos[1] = pos
    i = i + 1

    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()


    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = m.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

# print('test')
# renderer = mujoco.Renderer(model)
# mujoco.mj_forward(model, data)
# renderer.update_scene(data)
# media.show_image(renderer.render())


Installing mediapy:
-1.0
-1.001
-1.0019999999999998
-1.0029999999999997
-1.0039999999999996
-1.0049999999999994
-1.0059999999999993
-1.0069999999999992
-1.0079999999999991
-1.008999999999999
-1.009999999999999
-1.0109999999999988
-1.0119999999999987
-1.0129999999999986
-1.0139999999999985
-1.0149999999999983
-1.0159999999999982
-1.0169999999999981
-1.017999999999998
-1.018999999999998
-1.0199999999999978
-1.0209999999999977
-1.0219999999999976
-1.0229999999999975
-1.0239999999999974
-1.0249999999999972
-1.0259999999999971
-1.026999999999997
-1.027999999999997
-1.0289999999999968
-1.0299999999999967
-1.0309999999999966
-1.0319999999999965
-1.0329999999999964
-1.0339999999999963
-1.0349999999999961
-1.035999999999996
-1.036999999999996
-1.0379999999999958
-1.0389999999999957
-1.0399999999999956
-1.0409999999999955
-1.0419999999999954
-1.0429999999999953
-1.0439999999999952
-1.044999999999995
-1.045999999999995
-1.0469999999999948
-1.0479999999999947
-1.0489999999999946
-1.049999999999994

In [2]:
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/50_mesa.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

/tmp/ipykernel_31000/3780323363.py:1: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.util


Tue Feb 20 15:47:46 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.06              Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1070        On  |   00000000:26:00.0  On |                  N/A |
|  0%   54C    P0             33W /  166W |    1639MiB /   8192MiB |     11%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----